In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

In [ ]:
from IPython.display import display, Javascript

display(Javascript('''
(function() {
    var interval = setInterval(function() {
        if (typeof google !== 'undefined' && google.translate && google.translate.TranslateElement) {
            clearInterval(interval);
            google.translate.TranslateElement = function() {};
            document.getElementById('google_translate_element')?.remove();
        }
    }, 1000);
})();
'''))

## Update repository

In [ ]:
! git pull

## Install libraries

In [ ]:
! pip install -U --quiet git+https://github.com/greentfrapp/lucent.git

## Add import path

In [ ]:
import os
import sys
import gc

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
del module_path

## Organize imports

In [ ]:
import multiprocessing
from pathlib import Path

In [ ]:
from collections import OrderedDict

In [ ]:
from functools import reduce

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from tqdm import tqdm

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST

In [ ]:
from src.lattmc.fca.utils import *
from src.lattmc.fca.data_utils import *
from src.lattmc.fca.image_utils import *
from src.lattmc.fca.models import *
from src.lattmc.fca.fca_utils import *
from src.lattmc.fca.image_gens import *

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

In [ ]:
SEED = 2024

## Initialize Path

In [ ]:
PATH = Path('data')
model_dir = PATH / 'models'
model_path = model_dir / 'simple_cnn_mnist_model.ckpt'
config_dir = PATH / 'config'
config_dir.mkdir(exist_ok=True, parents=True)
config_1_layer_path = config_dir / 'neurons_1_layer.json'
config_cnn_layer_1 = config_dir / 'neurons_cnn_1_layer.json'
images_dir = PATH / 'images'
images_dir.mkdir(exist_ok=True, parents=True)
pumpkin_path = PATH / 'Pumpkin_Seeds_Dataset.xlsx'

## Load the model

In [ ]:
model = torch.load(model_path, map_location='cpu')

In [ ]:
def clear_state_dict(state_dict):
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.0.', 'conv1.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.3.', 'conv2.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.8.', 'fc1.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.11.', 'fc2.')] = state_dict.pop(key)

    return state_dict

In [ ]:
state_dict = clear_state_dict(model['state_dict'])

In [ ]:
net = nn.Sequential(OrderedDict([
    ('conv1', nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)),
    ('act1', nn.ReLU()),
    ('mxp1', nn.MaxPool2d(kernel_size=2, stride=2)),
    ('conv2', nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)),
    ('act2', nn.ReLU()),
    ('mxp2', nn.MaxPool2d(kernel_size=2, stride=2)),
    ('flatten', nn.Flatten()),
    ('fc1', nn.Linear(64 * 7 * 7, 128)),
    ('act3', nn.ReLU()),
    ('fc2', nn.Linear(128, 10)),
]))

In [ ]:
net.load_state_dict(state_dict)

In [ ]:
net = net.eval()

## Initialize MNIST dataset

In [ ]:
transforms_list = [
                ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)),
            ]

In [ ]:
transform = transforms.Compose(transforms_list)

In [ ]:
data_train = MNIST(images_dir, train=True, download=True)
data_test = MNIST(images_dir, train=False, download=True)

In [ ]:
next(net.parameters()).device

In [ ]:
device = find_device()
device

In [ ]:
wnet = NetWrapper(net, transform)

In [ ]:
wnet.net

In [ ]:
wnet.net[:6]

In [ ]:
layer_V_n = 3
layer_U_n = 6

In [ ]:
wnet.device

## Try feature visualization

In [ ]:
from lucent.optvis import render, param, transform, objectives
from lucent.misc.io.showing import animate_sequence
from lucent.modelzoo.util import get_model_layers

In [ ]:
from torchvision.models import googlenet
from lucent.modelzoo import inceptionv1

In [ ]:
device = 'cpu'

In [ ]:
# We have already downloaded the weights to inception5h.pth
inceptionv1_model = inceptionv1(pretrained=True)
_ = inceptionv1_model.to(device).eval()

In [ ]:
# Print the first 20 layer names
get_model_layers(inceptionv1_model)[:20]

In [ ]:
_ = render.render_vis(inceptionv1_model, "mixed4a:107", show_inline=True)

## Try feature visualization

In [ ]:
from lucent.optvis import render, param, transform as trsfm, objectives
from lucent.optvis.render import hook_model, tensor_to_img_array
from lucent.misc.io.showing import animate_sequence

In [ ]:
def render_vis(
    model,
    objective_f,
    param_f=None,
    optimizer=None,
    transforms=None,
    thresholds=(512,),
    verbose=False,
    preprocess=True,
    progress=True,
    show_image=True,
    save_image=False,
    image_name=None,
    show_inline=False,
    fixed_image_size=None,
):
    if param_f is None:
        param_f = lambda: param.image(128, device=next(model.parameters()).device)
    # param_f is a function that should return two things
    # params - parameters to update, which we pass to the optimizer
    # image_f - a function that returns an image as a tensor
    params, image_f = param_f()

    if optimizer is None:
        optimizer = lambda params: torch.optim.Adam(params, lr=5e-2)
    optimizer = optimizer(params)

    if transforms is None:
        transforms = transform.standard_transforms

    if preprocess:
        if model._get_name() == "InceptionV1":
            # Original Tensorflow InceptionV1 takes input range [-117, 138]
            transforms.append(transform.preprocess_inceptionv1())
        else:
            # Assume we use normalization for torchvision.models
            # See https://pytorch.org/docs/stable/torchvision/models.html
            transforms.append(transform.normalize())

    # Upsample images smaller than 224
    image_shape = image_f().shape
    if fixed_image_size is not None:
        new_size = fixed_image_size
    elif image_shape[2] < 224 or image_shape[3] < 224:
        new_size = 224
    else:
        new_size = None
    if new_size:
        transforms.append(
            torch.nn.Upsample(size=new_size, mode="bilinear", align_corners=True)
        )

    transform_f = trsfm.compose(transforms)

    hook, features = hook_model(model, image_f, return_hooks=True)
    objective_f = objectives.as_objective(objective_f)

    if verbose:
        model(transform_f(image_f()))
        print("Initial loss: {:.3f}".format(objective_f(hook)))

    images = []
    try:
        for i in tqdm(range(1, max(thresholds) + 1), disable=(not progress)):
            def closure():
                optimizer.zero_grad()
                try:
                    model(transform_f(image_f()))
                except RuntimeError as ex:
                    if i == 1:
                        # Only display the warning message
                        # on the first iteration, no need to do that
                        # every iteration
                        warnings.warn(
                            "Some layers could not be computed because the size of the "
                            "image is not big enough. It is fine, as long as the non"
                            "computed layers are not used in the objective function"
                            f"(exception details: '{ex}')"
                        )
                loss = objective_f(hook)
                loss.backward()
                return loss
                
            optimizer.step(closure)
            if i in thresholds:
                # image = tensor_to_img_array(image_f())
                if verbose:
                    print("Loss at step {}: {:.3f}".format(i, objective_f(hook)))
                    if show_inline:
                        show(image)
                images.append(image_f())
    except KeyboardInterrupt:
        print("Interrupted optimization at step {:d}.".format(i))
        if verbose:
            print("Loss at step {}: {:.3f}".format(i, objective_f(hook)))
        images.append(tensor_to_img_array(image_f()))

    # Clear hooks
    for module_hook in features.values():
        del module_hook.module._forward_hooks[module_hook.hook.id]

    if save_image:
        export(image_f(), image_name)
    if show_inline:
        show(tensor_to_img_array(image_f()))
    elif show_image:
        view(image_f())
    
    return images

In [ ]:
img_list = list()
chn_list = list()
laner_name = 'mxp1'
for ch in range(32):
    param_f = lambda: param.image(28, batch=16, channels=1)
    fft_decor_param_f = lambda: param.image(
        w=28, 
        fft=True, 
        decorrelate=True, 
        channels=1,
        batch=1,
    )
    obj = objectives.channel(
        laner_name, 
        ch, 
        batch=1
    ) - objectives.channel(
        laner_name, 
        ch, 
        batch=0
    )
    vnet = wnet.net.to('cpu').eval()
    thresholds=[2**k for k in range(9, 15)]
    results = render_vis(
        vnet, 
        obj,
        fft_decor_param_f,
        # param_f,
        # thresholds=thresholds,
        transforms=transforms_list,  
        fixed_image_size=28, 
        preprocess=False,
        save_image=False,
        show_inline=False,
        show_image=False,
    )
    img_list.append(results[0][0][0].to('cpu').detach().numpy())
    chn_list.append(str(ch + 1))

In [ ]:
len(results), results[0].shape, results[0][0][0].shape

In [ ]:
results[0].shape

In [ ]:
plt.imshow(results[0][0][0].to('cpu').detach().numpy())
plt.show()

In [ ]:
show_images([img[0] for img in results[0].detach().numpy()], [str(i + 1) for i in range(results[0].shape[0])])

In [ ]:
show_images(img_list, chn_list)

In [ ]:
show_images(img_list, chn_list)

## Get V lattices

In [ ]:
bs = 8

In [ ]:
V_X_train, X_V_train = layer_V(data_train, wnet, k=layer_V_n, bs=bs)

In [ ]:
V_X_test, X_V_test = layer_V(data_test, wnet, k=layer_V_n, bs=bs)

In [ ]:
U_X_train, X_U_train = layer_V(data_train, wnet, k=layer_U_n, bs=bs)

In [ ]:
U_X_test, X_U_test = layer_V(data_test, wnet, k=layer_U_n, bs=bs)

In [ ]:
V_X_train.shape, V_X_test.shape, U_X_train.shape, U_X_test.shape

## Sorting vectors

In [ ]:
V_X_digits, V_X_sorteds = sort_V_X(V_X_train, data_train)

## Alanyze maximum stimulus

In [ ]:
v_Ds = dict()
u_Ds = dict()
G_v_tests = dict()
G_u_tests = dict()
uncn_dict = dict()

In [ ]:
i = 0
ths = [
    328, #0
    280, #1
    320, #2
    384, #3
    300, #4
    300, #5
    400, #6
    200, #7
    380, #8
    180  #9
]
v = np.copy(V_X_sorteds[i][ths[i]])

In [ ]:
for i in range(10):
    layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
    G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
        v, 
        V_X_test, 
        U_X_test, 
        data_test
    )
    v_Ds[i] = v_D
    u_Ds[i] = u_D
    G_v_tests[i] = G_v_test
    G_u_tests[i] = G_u_test
    uncn_dict[i] = uncn_reps

In [ ]:
uncn_dict[i]

In [ ]:
show_grid(G_v_tests[i], data_test, nrow=32)

In [ ]:
show_grid(G_u_tests[i], data_test, nrow=32)

In [ ]:
y_hs = [np.argmax(wnet(data_test[idx][0])) for idx in G_u_test]

In [ ]:
uncn_hat = layer_fca.count_ys(y_hs)

In [ ]:
show_grid(G_u_tests[i], data_test, nrow=48, h=64, w=64, my=i)

## Experiments with shapes

In [ ]:
# Example usage
dataset = CustomShapeDataset(num_samples=1000)
# dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

shapes_shp = get_digits(dataset)

data_digits = get_digits(data_train)

# Get a batch of images
images, labels = zip(*[(x, y) for idx_l, (x, y) in enumerate(dataset) if idx_l < 32])

asgn_max = True

# Show a batch of images
show_images(images[:16], labels[:16])

In [ ]:
ring = np.copy(shapes_shp['nring'][0][0].numpy())
ring[-13 :, :] = 0
plt.imshow(ring)

In [ ]:
v_X_shapes_c = wnet(ring, k=layer_V_n)
visualize_slices(v_X_shapes_c[0])

In [ ]:
c_idx = 0
max_index, _, v_c = set_v(v_X_shapes_c[c_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_c[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_c, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
v_X_shapes_v = np.array(wnet(*shapes_shp['vertical_line'], k=layer_V_n))
v_X_shapes_h = np.array(wnet(*shapes_shp['horizontal_line'], k=layer_V_n))
v_X_shapes_e = np.array(wnet(*shapes_shp['sring'], k=layer_V_n))
v_X_shapes_r = np.array(wnet(*shapes_shp['nring'], k=layer_V_n))
v_X_shapes_v.shape, v_X_shapes_h.shape, v_X_shapes_e.shape, v_X_shapes_r.shape

In [ ]:
i = 3

In [ ]:
show_img(shapes_shp['vertical_line'], i)

In [ ]:
visualize_slices(v_X_shapes_h[i])

In [ ]:
diag = torch.zeros((28, 28), dtype=torch.float32)
for i in range(8, 24):
    diag[28 - i, i] = 255
diag /= 255
plt.imshow(diag)

In [ ]:
v_X_shapes_d = wnet(diag.unsqueeze(0), k=layer_V_n)

In [ ]:
visualize_slices(v_X_shapes_d[0])

In [ ]:
d_idx = 0
max_index, _, v_d = set_v(v_X_shapes_d[d_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_d[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_d, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=64)

In [ ]:
visualize_slices(v_X_shapes_v[0])

In [ ]:
v_idx = 0
max_index, _, v_v = set_v(v_X_shapes_v[v_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_v[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_v, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
visualize_slices(v_X_shapes_h[0])

In [ ]:
h_idx = 0
max_index, _, v_h = set_v(v_X_shapes_h[h_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_h[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_h, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
v_l = [v_v, v_h]
v = np.max(np.array(v_l), axis=0)
v.shape

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
visualize_slices(v_X_shapes_r[i])

In [ ]:
r_idx = 0
max_index, _, v_r = set_v(v_X_shapes_r[r_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_r[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_r, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
e_idx = 0
max_index, _, v_e = set_v(v_X_shapes_e[e_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_e[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_e, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
v_l = [
    # v_v, 
    v_h, 
    v_r, 
    v_d
]
v = np.max(np.array(v_l), axis=0)
v.shape

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=64)

In [ ]:
show(data_digits[0][0:13])

In [ ]:
img_indices = [0, 4, 7, 10]
show([data_digits[0][k] for k in img_indices])

In [ ]:
zr_idx = 2
max_index, _, v_0 = set_vs(*V_X_digits[0][img_indices], denm=1.61, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_z[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_0, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=64)

In [ ]:
show(data_digits[1][0:13])

In [ ]:
img_indices = [0, 1, 5, 9]
show([data_digits[1][k] for k in img_indices])

In [ ]:
on_idx = 2
max_index, _, v_1 = set_vs(*V_X_digits[1][img_indices], denm=1.438, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_o[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_1, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=64)

In [ ]:
show(data_digits[2][13:26])

In [ ]:
img_indices = [0, 3, 4, 5, 7, 16, 17, 18, 19, 20, 23, 24, 25]
show([data_digits[2][k] for k in img_indices])

In [ ]:
tw_idx = 2
max_index, _, v_2 = set_vs(*V_X_digits[2][img_indices], denm=2.2, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_t[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_2, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=32)

In [ ]:
show(data_digits[3][0:13])

In [ ]:
img_indices = [0, 1, 6, 12]
show([data_digits[3][k] for k in img_indices])

In [ ]:
ht_idx = 3
max_index, _, v_3 = set_vs(*V_X_digits[3][img_indices], denm=2.2, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_3[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_3, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=64)

In [ ]:
show(data_digits[4][0:13])

In [ ]:
img_indices = [0, 1, 3, 4, 5, 6, 7, 8]
show([data_digits[4][k] for k in img_indices])

In [ ]:
fr_idx = 2
max_index, _, v_4 = set_vs(*V_X_digits[4][img_indices], denm=4.1, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_4[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_4, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=32)

In [ ]:
show(data_digits[5][13:26])

In [ ]:
img_indices = [0, 1, 7, 10, 11, 16, 17]
show([data_digits[5][k] for k in img_indices])

In [ ]:
fv_idx = 2
max_index, _, v_5 = set_vs(*V_X_digits[5][img_indices], denm=3, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_5[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_5, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=32)

In [ ]:
show(data_digits[6][0:13])

In [ ]:
img_indices = [2, 5, 14, 17, 20]
show([data_digits[6][k] for k in img_indices])

In [ ]:
sx_idx = 2
max_index, _, v_6 = set_vs(*V_X_digits[6][img_indices], denm=2.2, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_6[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_6, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=32)

In [ ]:
show(data_digits[7][0:13])

In [ ]:
img_indices = list(range(13))
show([data_digits[7][k] for k in img_indices])

In [ ]:
sv_idx = 2
max_index, _, v_7 = set_vs(*V_X_digits[7][img_indices], denm=4, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_7[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_7, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=32)

In [ ]:
show(data_digits[8][26:39])

In [ ]:
img_indices = [4, 12, 21, 33, 35]
show([data_digits[8][k] for k in img_indices])

In [ ]:
eg_idx = 2
max_index, _, v_8 = set_vs(*V_X_digits[8][img_indices], denm=1.4, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_8[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_8, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=32)

In [ ]:
show(data_digits[9][0:13])

In [ ]:
img_indices = [1, 4, 5, 7, 13]
show([data_digits[9][k] for k in img_indices])

In [ ]:
n_idx = 2
max_index, _, v_9 = set_vs(*V_X_digits[9][img_indices], denm=2, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_n[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_9, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=64)